In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

#import NetGeometry as ng
# No
import NlGeometry as ng

Инициализация объектов PointBase - точечный базис с известными метрическими тензорами.

In [2]:
# Прямоугольный треугольник
lPoints = [[0, 0], [0, 3], [4, 0]] # координаты вершин в декартовой системе
mDist = ng.Matrix.mDistance(lPoints) # дистанционная матрица (квадраты расстояний)
pbRightTriangle = ng.BaseSet(mDist)

# Граф из Википедии
lConnects = [[1, 2], [2, 3], [3, 4], [4, 5], [4, 6], [1, 5], [2, 5]] # список ребер графа
mL = ng.Matrix.mtrLaplacian(ng.Matrix(lConnects).matrix) # лапласиан
pbWikigraph = ng.BaseSet(mL, 'Laplacian')

# Графы заданной топологии
n, c = 5, 1 # количество узлов и вес ребра (сила связи, проводимость)
pbFull = ng.FullGraph(n, c) # полный граф
pbCloseChain = ng.ChainGraph(n, c, True) # замкнутая цепь
pbOpenChain = ng.ChainGraph(n, c, False) # разомкнутая цепь
pbStar = ng.StarGraph(n, c) # граф-звезда


# Свойства базисов

Метрические тензоры

In [3]:
print(pbRightTriangle.Gm)
print()
print(pbRightTriangle.Lm)

[[  0.    1.    1.    1. ]
 [  1.    0.   -4.5  -8. ]
 [  1.   -4.5   0.  -12.5]
 [  1.   -8.  -12.5   0. ]]

[[ 6.25        0.          0.5         0.5       ]
 [-0.          0.17361111 -0.11111111 -0.0625    ]
 [ 0.5        -0.11111111  0.11111111  0.        ]
 [ 0.5        -0.0625      0.          0.0625    ]]


Радиус сферы (окружности) и объем симплекса (здесь - площадь треугольника)

In [4]:
pbRightTriangle.ro, pbRightTriangle.volume()

(6.25, 6.0)

## Индекс симметричности

Индекс симметричности любого прямоугольного треугольника равен 8/9:

In [17]:
pbRightTriangle.symIndex(), 8/9

(0.88888888888888884, 0.8888888888888888)

Индекс симметричности полного графа и замкнутой цепочки равны 1:

In [7]:
pbFull.symIndex(), pbCloseChain.symIndex()

(1.0, 1.0)

Индекс симметричности графа-звезды выражается как:

$is(n)=4(n-1)/n^2$

In [8]:
pbStar.symIndex(), 4*(n-1)/n**2

(0.64000000000000001, 0.64)

Для индекса симметричности разомкнутой цепи имеет место следующее выражение:

$is(n) = 2(n+1)/3n$

In [9]:
pbOpenChain.symIndex(), 2*(n+1)/(3*n)

(0.79999999999999993, 0.8)

## Ортогональная связность базиса

Для полного графа (все узлы связаны со всеми с одной и той же силой связи $c$) выражение для сферической связности $\chi$ как функции количества узлов графа $n$ имеет вид: 

$\chi_{max}(n) = c \frac{n^2}{n-1}$

In [18]:
pbFull.connectivity(), c*n**2/(n-1)

(6.25, 6.25)

Минимальная связность при фиксированном количестве узлов $n$ - это связность разомкнутой цепочки или звезды с силой связи $c$ между звеньями:

$\chi_{min}(n)=c \frac{4}{n-1}$

In [11]:
pbStar.connectivity(), pbOpenChain.connectivity(), 4*c/(n-1)

(1.0, 1.0000000000000004, 1.0)

Ортогональная связность замкнутой цепочки:

$\chi_c(n) = 12c \frac{n}{n^2-1}$

In [12]:
pbCloseChain.connectivity(), 12*c*n/(n**2-1)

(2.5, 2.5)

## Свойства графа из Википедии


<img src="pictures/6n-graf_s.png">

Связность и симметричность:

In [13]:
pbWikigraph.connectivity(), pbWikigraph.symIndex()

(1.6632302405498278, 0.77281405116456658)

Скалярный потенциал равен 11 - это количество возможных остовов.

In [14]:
pbWikigraph.cf

10.999999999999996

Барицентрические координаты центра сферы, удаленность узлов:

In [15]:
print(pbWikigraph.bcCo)

[ 0.36363636  0.04545455  0.27272727 -0.22727273  0.04545455  0.5       ]


### Свойства wheels & fans

In [16]:
def Fraction(val, limit=1000, acc=0.0000000001):
    for i in range(1, limit+1):
        ch = val*i
        if abs(ch - round(ch)) < acc:
            return (round(ch), i)
    return (0, 0)

#fr = 0.11522443632496149
#Fraction(fr, 10000000, 0.0000001)

def pbWheels(numArms=5, closed=True):
    lConnects = []
    for i in range(2, numArms+2):
        lConnects.append([1, i])
        j = i+1
        if j > numArms+1:
            if closed: lConnects.append([i, 2])
        else:
            lConnects.append([i, j])
    mL = ng.Matrix.mtrLaplacian(ng.Matrix(lConnects).matrix) # лапласиан
    return ng.BaseSet(mL, 'Laplacian')

pbWheel = pbWheels(5, False) #False
#pbWheel.mL
#Fraction(pbWheel.bcSphere[1], 10000000, 0.0000001)
pbWheel.bcCo

#for n in range(21, 31):
#    pbWheel = pbWheels(n, False) #
#    print(n, ": ", pbWheel.uK, pbWheel.rs)

array([-0.31818182,  0.38181818,  0.17272727,  0.20909091,  0.17272727,
        0.38181818])

Wheels могут быть замкнутые и разомкнутые.
Замкнутые - есть центр, с которым соединены остальные вершины (как списцы колеса). 
Вершины также соединены между собой. В итоге кардинальность (степень) каждой вершины - 3 (кроме центральной).
Похожи на замкнутую цепь, но с наличием центра.

Разомкнутые - как замкнутые, только есть две крайних вершины (обода), между которыми нет связи.
Кардинальность крайних вершин - 2.

Есть явные выражения для скалярного потенциала (постоянной Кирхгофа) $u(n)$ таких графов.
Связаны с числами Фибоначчи $F(n)$. Здесь и далее $(n)$ - количество точек обода (на 1 меньше общего количества вершин).

Для разомкнутого колеса:
$u(n) = F(2n)$

Для замкнутого зависит от четности $n$. Условие $n > 2$.

Четные: $u(n) = 5F(n)^2 = 5(F(n-1)F(n+1) - 1)$

Нечетные: $uK(n) = 5 F(n)^2 - 4 = 5 F(n-1)F(n+1) + 1$

Сложнее с определением связности (обратного радиуса) $w=1/rs$.

Для замкнутого колеса связность связана со скалярным потенциалом таким образом:

$w(n) = 20u(n) / n(4 + u(n))$

Для разомкнутого есть пока только некое приближенное соотношение:

$w = 20/(n + 20A)$, где $A = 0.11522443632..$ - непонятная константа, которую хорошо бы распознать.

Точность довольно высока. Погрешность для n=3 составляет всего 0.151%, для n=6 - 0.001%, дальше еще меньше.

Другой подоход - через скалярный потенциал.

$w(n) = 4u(n)^2/K(n)$, где $K(n)$ - некое целое число. 

Но тут проблема в определении последовательности $K(n)$. Первые числа такие (с n=2):
8, 68, 556, 4419, 34440, 264467, и т.д. Непонятно, что за цифры. В энциклопедии такой последовательности нет.


In [32]:
norms = np.array([1, 2, 3])
ones = np.ones(3)
np.outer(ones, norms)

array([[ 1.,  2.,  3.],
       [ 1.,  2.,  3.],
       [ 1.,  2.,  3.]])